In [1]:
import os
import io
import random
import numpy as np
import pandas as pd
from PIL import Image
import autokeras as ak
import tensorflow as tf
from numpy import asarray
from tensorflow import keras
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from tensorflow.keras.callbacks import CSVLogger
from sklearn.datasets import load_files
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

In [3]:
#create an image array from all instances of the three domains csv with instances labels, label per solver

directory = ["All-6Set/CHRImages","All-6Set/GREEDYImages","All-6Set/EXPImages","All-6Set/COMBOImages","All-6Set/DSATURImages","All-6Set/LFImages"]
images = []
total = 0
Label = 0
df = pd.DataFrame(columns = ['instance', 'Label'])
for i in directory:
    for filename in os.listdir(i):
        instance = os.path.join(i, filename)
        total+=1
        df.loc[total] = [filename,Label]
        if instance.endswith('.jpg'):
            im = Image.open(instance).convert('L')
            pix = im.load()
            width, height = im.size
            pixel_values = list(im.getdata())
            data = asarray(im)
            images.append(data)
    Label+=1
    
images2 = np.asarray(images)
images2 = images2.reshape((images2.shape[0], 32, 32))
df.to_csv('allsolvers.csv') 

In [ ]:
#read the labels
npdf = pd.read_csv('allsolvers.csv')
print(npdf['Label'].value_counts())
source = npdf.Label

In [ ]:
#normalise the values between 0-1
X = images2.astype('float32')
X /= 255
print('X_train shape:', X.shape)
y = source

In [6]:
tf_callbacks = [
    tf.keras.callbacks.CSVLogger("TSPimagesTraining.csv", separator=",", append=False),
    tf.keras.callbacks.TensorBoard(log_dir='./logs',profile_batch = 100000000),
]

In [ ]:
# Test Autokeras Model
clf = ak.ImageClassifier(overwrite=False, max_trials=1)
# Supervised training of the model
clf.fit(X, y, epochs=2, callbacks=tf_callbacks)

In [ ]:
#export the selected model to check network architecture
model = clf.export_model()
model.summary()

# Autkeras Model

In [8]:
from sklearn import metrics
from sklearn.metrics import accuracy_score,recall_score,precision_score
cv = KFold(n_splits=10, random_state=1, shuffle=True)
all_tests = []
all_predictions = []
for train_index, test_index in cv.split(X, y):
  #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf = ak.ImageClassifier(overwrite= False, max_trials=1)
  # Supervised training of the model
    print("Start training")
    clf.fit(X_train, y_train, epochs=100, callbacks=tf_callbacks)

    print("Predictions on unseen data")
    predicted_y = clf.predict(X_test)
    all_tests.append(y_test)
    all_predictions.append(predicted_y)
    report = classification_report(y_test.astype(str), predicted_y.astype(str))
    print(report)

INFO:tensorflow:Reloading Oracle from existing project .\image_classifier\oracle.json
INFO:tensorflow:Reloading Tuner from .\image_classifier\tuner0.json
Start training
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
422/422 [==============================] - 13s 30ms/step - loss: 1.6744 - accuracy: 0.7678
Epoch 2/100
422/422 [==============================] - 13s 30ms/step - loss: 1.6233 - accuracy: 0.7992
Epoch 3/100
422/422 [==============================] - 13s 30ms/step - loss: 0.9579 - accuracy: 0.84080s - loss: 0.9881 - ac
Epoch 4/100
422/422 [==============================] - 13s 31ms/step - loss: 1.3986 - accuracy: 0.8243
Epoch 5/100
422/422 [==============================] - 13s 30ms/step - loss: 0.8110 - accuracy: 0.8742
Epoch 6/100
422/422 [==============================] - 13s 30ms/step - loss: 1.2245 - accuracy: 0.8556
Epoch 7/100
422/422 [==============================] - 13s 31ms/step - loss: 0.8870 - accuracy: 0.8540
Epoch 8/100
422/422 [=============================

422/422 [==============================] - 13s 31ms/step - loss: 0.4281 - accuracy: 0.9056
Epoch 48/100
422/422 [==============================] - 13s 31ms/step - loss: 0.7691 - accuracy: 0.8321
Epoch 49/100
422/422 [==============================] - 13s 31ms/step - loss: 0.6338 - accuracy: 0.7831
Epoch 50/100
422/422 [==============================] - 13s 31ms/step - loss: 0.6338 - accuracy: 0.7839
Epoch 51/100
422/422 [==============================] - 13s 31ms/step - loss: 0.4753 - accuracy: 0.8644
Epoch 52/100
422/422 [==============================] - 13s 31ms/step - loss: 0.6056 - accuracy: 0.8690
Epoch 53/100
422/422 [==============================] - 13s 31ms/step - loss: 0.5321 - accuracy: 0.8927
Epoch 54/100
422/422 [==============================] - 13s 31ms/step - loss: 0.5788 - accuracy: 0.8826
Epoch 55/100
422/422 [==============================] - 13s 31ms/step - loss: 0.6082 - accuracy: 0.8773
Epoch 56/100
422/422 [==============================] - 13s 31ms/step - loss:

422/422 [==============================] - 13s 30ms/step - loss: 1.0636 - accuracy: 0.9020
Epoch 96/100
422/422 [==============================] - 13s 30ms/step - loss: 1.0199 - accuracy: 0.8499
Epoch 97/100
422/422 [==============================] - 13s 30ms/step - loss: 0.8066 - accuracy: 0.8262
Epoch 98/100
422/422 [==============================] - 13s 30ms/step - loss: 0.6131 - accuracy: 0.8311
Epoch 99/100
422/422 [==============================] - 13s 30ms/step - loss: 0.7570 - accuracy: 0.8196
Epoch 100/100
422/422 [==============================] - 13s 30ms/step - loss: 0.6250 - accuracy: 0.7890
INFO:tensorflow:Assets written to: .\image_classifier\best_model\assets
Predictions on unseen data
47/47 [==============================] - 0s 7ms/step
              precision    recall  f1-score   support

           0       0.80      0.02      0.03       253
           1       0.50      1.00      0.66       247
           2       0.99      0.60      0.74       243
           3       

422/422 [==============================] - 13s 30ms/step - loss: 0.7426 - accuracy: 0.8642
Epoch 36/100
422/422 [==============================] - 13s 30ms/step - loss: 1.2950 - accuracy: 0.9030
Epoch 37/100
422/422 [==============================] - 13s 30ms/step - loss: 0.7718 - accuracy: 0.8033
Epoch 38/100
422/422 [==============================] - 13s 30ms/step - loss: 0.6314 - accuracy: 0.8071
Epoch 39/100
422/422 [==============================] - 13s 30ms/step - loss: 0.5994 - accuracy: 0.8309
Epoch 40/100
422/422 [==============================] - 13s 30ms/step - loss: 0.5768 - accuracy: 0.8522
Epoch 41/100
422/422 [==============================] - 13s 30ms/step - loss: 0.6703 - accuracy: 0.8330
Epoch 42/100
422/422 [==============================] - 13s 30ms/step - loss: 0.8618 - accuracy: 0.8831
Epoch 43/100
422/422 [==============================] - 13s 30ms/step - loss: 0.7238 - accuracy: 0.9024
Epoch 44/100
422/422 [==============================] - 13s 30ms/step - loss:

422/422 [==============================] - 13s 30ms/step - loss: 0.6507 - accuracy: 0.7364
Epoch 84/100
422/422 [==============================] - 13s 30ms/step - loss: 0.6781 - accuracy: 0.7921
Epoch 85/100
422/422 [==============================] - 13s 30ms/step - loss: 0.6885 - accuracy: 0.82930s - loss: 0.7092 
Epoch 86/100
422/422 [==============================] - 13s 30ms/step - loss: 0.6083 - accuracy: 0.7781
Epoch 87/100
422/422 [==============================] - 13s 30ms/step - loss: 0.5309 - accuracy: 0.7873
Epoch 88/100
422/422 [==============================] - 13s 30ms/step - loss: 0.5980 - accuracy: 0.7522
Epoch 89/100
422/422 [==============================] - 13s 30ms/step - loss: 0.4967 - accuracy: 0.8182
Epoch 90/100
422/422 [==============================] - 13s 31ms/step - loss: 0.6082 - accuracy: 0.8500
Epoch 91/100
422/422 [==============================] - 13s 30ms/step - loss: 0.6005 - accuracy: 0.8260
Epoch 92/100
422/422 [==============================] - 13s

C:\Users\40016092\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\40016092\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\40016092\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:tensorflow:Oracle triggered exit
Epoch 1/100
422/422 [==============================] - 13s 30ms/step - loss: 1.6729 - accuracy: 0.7624
Epoch 2/100
422/422 [==============================] - 13s 30ms/step - loss: 1.7070 - accuracy: 0.6780
Epoch 3/100
422/422 [==============================] - 13s 30ms/step - loss: 1.6771 - accuracy: 0.7496
Epoch 4/100
422/422 [==============================] - 13s 30ms/step - loss: 1.4615 - accuracy: 0.7825
Epoch 5/100
422/422 [==============================] - 13s 30ms/step - loss: 1.0086 - accuracy: 0.7867
Epoch 6/100
422/422 [==============================] - 13s 30ms/step - loss: 0.9966 - accuracy: 0.8472
Epoch 7/100
422/422 [==============================] - 13s 30ms/step - loss: 1.1755 - accuracy: 0.7741
Epoch 8/100
422/422 [==============================] - 13s 30ms/step - loss: 1.2259 - accuracy: 0.8151
Epoch 9/100
422/422 [==============================] - 13s 30ms/step - loss: 0.8675 - accuracy: 0.8656
Epoch 10/100
422/422 [=============

422/422 [==============================] - 13s 30ms/step - loss: 0.8329 - accuracy: 0.8632
Epoch 50/100
422/422 [==============================] - 13s 30ms/step - loss: 0.7575 - accuracy: 0.8346
Epoch 51/100
422/422 [==============================] - 13s 30ms/step - loss: 0.9006 - accuracy: 0.7507
Epoch 52/100
422/422 [==============================] - 13s 30ms/step - loss: 0.6118 - accuracy: 0.8521
Epoch 53/100
422/422 [==============================] - 13s 30ms/step - loss: 0.6770 - accuracy: 0.8416
Epoch 54/100
422/422 [==============================] - 13s 30ms/step - loss: 1.1234 - accuracy: 0.8596
Epoch 55/100
422/422 [==============================] - 13s 30ms/step - loss: 0.5791 - accuracy: 0.8507
Epoch 56/100
422/422 [==============================] - 13s 30ms/step - loss: 0.6485 - accuracy: 0.8716
Epoch 57/100
422/422 [==============================] - 13s 30ms/step - loss: 0.7222 - accuracy: 0.7654
Epoch 58/100
422/422 [==============================] - 13s 30ms/step - loss:

422/422 [==============================] - 13s 30ms/step - loss: 0.4623 - accuracy: 0.8710
Epoch 98/100
422/422 [==============================] - 13s 30ms/step - loss: 0.5853 - accuracy: 0.8655
Epoch 99/100
422/422 [==============================] - 13s 30ms/step - loss: 0.7970 - accuracy: 0.8032
Epoch 100/100
422/422 [==============================] - 13s 30ms/step - loss: 0.6560 - accuracy: 0.7687
INFO:tensorflow:Assets written to: .\image_classifier\best_model\assets
Predictions on unseen data
47/47 [==============================] - 0s 6ms/step
              precision    recall  f1-score   support

           0       1.00      0.04      0.08       241
           1       0.51      1.00      0.68       244
           2       0.99      0.46      0.63       244
           3       0.64      1.00      0.78       238
           4       0.72      0.69      0.70       272
           5       0.69      0.72      0.70       261

    accuracy                           0.65      1500
   macro a

In [25]:
predictions = []
for sublist in all_predictions:
    for item in sublist:
        predictions.append(int(item))

tests = []
for sublist in all_tests:
    for item in sublist:
        tests.append(item)

In [26]:
report = classification_report(tests,predictions)
print(report)

              precision    recall  f1-score   support

           0       0.73      0.06      0.11      2500
           1       0.50      0.98      0.67      2500
           2       0.96      0.51      0.67      2500
           3       0.68      0.97      0.79      2500
           4       0.73      0.27      0.39      2500
           5       0.55      0.90      0.68      2500

    accuracy                           0.61     15000
   macro avg       0.69      0.61      0.55     15000
weighted avg       0.69      0.61      0.55     15000



In [27]:
from sklearn import metrics
from sklearn.metrics import accuracy_score,recall_score,precision_score
from sklearn.metrics import f1_score

print("Accuracy:",metrics.accuracy_score(tests,predictions))
precision = precision_score(predictions,tests,average="macro")
recall = recall_score(predictions,tests,average="macro")
f1_score = f1_score(predictions,tests,average="macro")
print('Precision: %.3f' % precision)
print('Recall: %.3f' % recall)
print('F1-score: %.3f' % f1_score)

Accuracy: 0.6145333333333334
Precision: 0.615
Recall: 0.691
F1-score: 0.552
